# Der Transistor

In [ ]:
#| code-fold: true


#import sys
# adding Folder_2 to the system path
#sys.path.insert(0, '../../../Common/QuartoBookHelpers')
#import QuartoBookHelpers

from IPython.display import Markdown, Latex

from sympy import *
import sympy.physics.units as u
import numpy as np
from matplotlib import pyplot as plt
from quantiphy import Quantity
import re
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal

class MySymbol(Symbol):
    def __new__(cls, name, description='',unit='',value='',**kwargs):
        obj = Symbol.__new__(cls, name,**kwargs)
        obj.description = description
        obj.unit = unit
        obj.value = value
        
        return obj

class MyFunction(Function):
    def __new__(cls, name, description='',unit='',value=''):
        obj = Function.__new__(cls, name)
        obj.description = description
        obj.unit = unit
        obj.value = value
        
        return obj
        
class QBookHelpers():
    def print_description(dic):
        for vari in dic:
            display(Markdown(f'${latex(vari)}$' + ' ... ' + vari.description))
            
    def print_equation(eq,**kwargs):
        #print(kwargs)
        if kwargs != {}:
            label = kwargs['label']
            #print(label)
            display(Markdown(f'$$\n{latex(eq)}\n$${{#' + label + f'}}'))
        else:      
            display(Markdown(f'$$\n{latex(eq)}\n$${{#eq-Dummy}}'))

    def print_values2(dic,**kwargs):
        for vari in dic:
            QuantValue = Quantity(vari.value)
            #print(QuantValue)
            quant_string = QuantValue.render(prec=2)
            #print(quant_string)
            val = re.split('(\d+)', quant_string)[1]
            if re.split('(\d+)', quant_string)[2] == '.':
                val = val + re.split('(\d+)', quant_string)[2] + re.split('(\d+)', quant_string)[3]
            #print(val)
            try:
                prefix = re.split('(\d+)', quant_string)[-1]
            except:
                prefix = ''
            #print(prefix)
            uni = vari.unit
            disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' \ ' + latex(prefix) + latex(uni) + f'$$' 
            display(Markdown(disp_string))
            
    def print_values(dic,**kwargs):
        for vari in dic:
            try:
                QuantValue = Quantity(vari.value,str(vari.unit.abbrev))
            except:
                QuantValue = Quantity(vari.value)
            quant_string = QuantValue.render(prec=2)
            val = quant_string.split(' ')[0]
            if quant_string[-1].isdigit():
                uni = ''
            else:
                try:
                    uni = quant_string.split(' ')[1]
                except:
                    uni = quant_string[-1]
            #print(re.split('(\d+)',QuantValue.render(prec=2)))
            #uni = re.split('(\d+)',QuantValue.render(prec=2))[-1]
            #uni = ''
            #val = re.split('(\d+)',QuantValue.render(prec=2))[1]
            #val = QuantValue.render(prec=2)

            if uni.find('ohm') >=0:
                uni = '\ \mathrm{' + uni.replace('ohm','\Omega') + '}'
                disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' ' + str(uni) + f'$$' 
            else:
                disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' \ ' + latex(uni) + f'$$' 
            
            if kwargs != {}:
                label = kwargs['label']
                disp_string = disp_string + f'{{#' + label + f'}}'

            
            display(Markdown(disp_string))

            #display(Markdown(f'${latex(vari)}$' + ' = ' + str(vari.value) + f'$\ {latex(vari.unit)}$' ))            
            #display(Markdown(f'${latex(vari)}$' + ' = ' + str("%.2f" % round(vari.value, 2)) + f'$\ {latex(vari.unit)}$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + latex(QuantValue.render()) + f'$$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + latex(quant) + f'$$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + str(vari.value) + f'\ {latex(vari.unit)}$$' ))

            
    def calculate_num_value(eqn):
        replacement = {}
        for sym in eqn.rhs.free_symbols:
            try:
                case = {sym:sym.value}
                replacement.update(case)
            except:
                pass
        #print(replacement)
        #return Eq(eqn.lhs,eqn.rhs.evalf(subs=replacement))
        eqn.lhs.value = eqn.rhs.evalf(subs=replacement)
        #display(eqn.lhs.value)

    def replace_num_value(eqn):
        replacement = {}
        for sym in eqn.rhs.free_symbols:
            try:
                case = {sym:sym.value}
                replacement.update(case)
            except:
                pass
        #print(replacement)
        try:
            eqn = Eq(eqn.lhs,eqn.rhs.evalf(subs=replacement))
        except:
            eqn = Eq(eqn.lhs,eqn.rhs.subs(subs=replacement))
        return eqn

    def logic_parser(eqn):
        return str(eqn).replace('&','and').replace('|','or').replace('~','not')

    def trans_plot_matplotlib(filepath, thing_to_plot):
        df = pd.read_csv(filepath,sep=';',decimal=',')
        for key in thing_to_plot:
            suplotnumber_max = 0
            if max(thing_to_plot[key]) > 0:
                suplotnumber_max = max(thing_to_plot[key])
        
        fig, ax = plt.subplots(suplotnumber_max,1,figsize=(10, 6*suplotnumber_max))
        for i, key in enumerate(thing_to_plot):
            #for i in thing_to_plot[key]:
            ax[i].plot(df.time, df[key],label=key)
            ax[i].set_xlabel('Time in s')
            ax[i].set_ylabel(key)
            ax[i].grid()
            ax[i].legend()

        plt.tight_layout()
        plt.savefig(filepath.replace('.csv','.svg'))
        plt.show()
        

    def trans_plot(filepath, thing_to_plot):
        df = pd.read_csv(filepath,sep=';',decimal=',')

        for key in thing_to_plot:
            suplotnumber_max = 0
            if max(thing_to_plot[key]) > 0:
                suplotnumber_max = max(thing_to_plot[key])

        fig = make_subplots(
            rows=suplotnumber_max, cols=1,
        )

        for key in thing_to_plot:
            for i in thing_to_plot[key]:
                fig.add_trace(go.Scatter(x=df.time.values, y=df[key].values,name=key), row=thing_to_plot[key][0], col=1)
                fig.update_xaxes(title_text=None, tickformat = ".3s", ticksuffix="s",  row=thing_to_plot[key][0], col=1)
                if key[0] == 'v':
                    ticksuff = 'V'
                elif key[0] == 'i':
                    ticksuff = 'A'
                else:
                    ticksuff = ''
                fig.update_yaxes(title=key, ticksuffix=ticksuff,tickformat = ".3s", row=thing_to_plot[key][0], col=1)

        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.15,
                xanchor="left",
                x=0.01
            ),
            height=1000/8*suplotnumber_max,
            width=800,
        )

        fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def plotly_plot(x,y, **kwargs):
        # if not np.isscalar(y):
        #     ys = y
        #     y = y[0]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='lines',hovertemplate=None))
        fig.update_layout(hovermode="x unified")
        fig.update_xaxes(title_text=None) #, tickformat = ".3s"
        fig.update_yaxes(title='y') #, tickformat = ".3s"
        for name in kwargs:
            if name == 'show_axis':
                if not kwargs[name]:
                    fig.update_xaxes(visible=False)
                    fig.update_yaxes(visible=False)
            if name == 'yrange':
                fig.update_yaxes(range=kwargs[name])
            if name == 'log_x':
                if kwargs[name]:
                    fig.update_xaxes(type="log")
            if name == 'log_y':
                if kwargs[name]:
                    fig.update_yaxes(type="log")
            if name == 'xticksuffix':
                fig.update_xaxes(ticksuffix=kwargs[name])
            if name == 'yticksuffix':
                fig.update_yaxes(ticksuffix=kwargs[name])
            if name == 'xtickformat':
                fig.update_xaxes(tickformat=kwargs[name])
            if name == 'ytickformat':
                fig.update_yaxes(tickformat=kwargs[name])
            if name == 'xtitle_text':
                fig.update_xaxes(title_text=kwargs[name])   
            if name == 'ytitle_text':
                fig.update_yaxes(title_text=kwargs[name])

        # if not np.isscalar(ys):
        #     for i in range(1,len(ys)):
        #         fig.add_trace(go.Scatter(x=x, y=ys[i], mode='lines', name='lines'))
        fig.show()

    def plotly_plot2(x,ys, **kwargs): #Started to plot multiple charts
        fig = make_subplots(rows=len(ys), cols=1)

        for i,y in enumerate(ys):
            fig.add_trace(go.Scatter(x=x, y=y, mode='lines',hovertemplate=None),row=i+1,col=1)
            fig.update_xaxes(type='log', row=i+1, col=1, title='Frequency in rad/s',tickformat = ".0e")

        fig.update_yaxes(title='Magnitude in dB', row=1, col=1)
        fig.update_yaxes(title='Phase in °', row=2, col=1)


        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
   #         height=1000/8*suplotnumber_max,
            width=800,
        )

 #       fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def plotly_plot3(x,ys, **kwargs): #Used for Bode Plot
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(go.Scatter(x=x, y=ys[0],name='Magnitude in dB', mode='lines'),secondary_y=False,)
        fig.add_trace(go.Scatter(x=x, y=ys[1],name='Phase in °', mode='lines'),secondary_y=True,)
        fig.update_xaxes(type='log', row=1, col=1, title='Frequency in rad/s',tickformat = ".0e")

        fig.update_yaxes(title_text='Magnitude in dB', secondary_y=False)
        fig.update_yaxes(title_text='Phase in °', secondary_y=True)


        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.15,
                xanchor="left",
                x=0.01
            ),
   #         height=1000/8*suplotnumber_max,
            width=800,
        )

 #       fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def symTF_to_sciTF(symTF):
        #convert sympy transfer function to scipy system
        if str(symTF.num).find('s') == -1:
            num = [symTF.num]
        else:
            num = Poly(symTF.num).all_coeffs()

        if str(symTF.den).find('s') == -1:
            den = [symTF.den]
        else:
            den = Poly(symTF.den).all_coeffs()

        #convert sympy numbers to float
        num = [float(i) for i in num]
        den = [float(i) for i in den]

        sciTF = signal.TransferFunction(num, den)
        return sciTF


## Feldeffekttransistor {#sec-MOSFET}

Die einfachste und die gleichzeitig eine der wichtigsten Anwendungen des MOSFET's ist der Schalter. Mittels Spannnung am Gate wird der MOSFET ein- und ausgeschalten.

![N-Kanal MOSFET als Schalter](Grafiken/NChannelenhancementSwitch){#fig-n-Channel_switch}

Die notwendigen und zulässigen Spannungen sind aus dem Datenblatt des gewählten Transistors zu entnehmen. Die kleinste Spannung, welche für das Schalten zwischen Gate und Source anliegen muss, wird Threshold-Spannung $U_{GS,Threshhold}$ genannt. Am Gate wird kein Vorwiderstand benötigt, da der Eingangswiderstand des MOSFET's sehr hoch ist und dadurch, in den meisten Fällen, mit $I_D = 0 \ \mathrm{A}$ angenommen werden kann.

### Aufgabe

#### Teil 1: N-Kanal Anreicherungstyp
Simulieren Sie die Schaltung. Wählen Sie die Spannungen aus dem Datenblatt aus. Geben Sie für zwei Eingangspulse den Strom durch, und die Spannung über den Widerstand.
Verwenden Sie dafür die Transientenanalyse und geben Sie deutlich an ob das Ergebnis den Erwartungen entspricht oder nicht. Argumentieren Sie Ihre Aussage.

#### Teil 2: P-Kanal Anreicherungstyp
Simulieren Sie die Schaltung erneut unter der verwendung eines P-Kanal Anreicherungstypen. Passen Sie die Spannungen so an, dass auch dieser als Schalter funktioniert. Verwenden Sie dazu erneut das passende Datenblatt.
Geben Sie deutlich an ob das Ergebnis den Erwartungen entspricht oder nicht. Argumentieren Sie Ihre Aussage.


## Bipolartransistor {#sec-BJT}

<!-- ![Bipolartransistor in Emittergrundschaltung](Grafiken/BJTEmitter_Basic){#fig-BJT_Emitter_Basic} -->


In [ ]:
#| code-fold: true
#| echo: false

R1 = MySymbol('R_1',description='Widerstand 1 am Spannungsteiler für die AP einstellung')
R2 = MySymbol('R_2')
Rc = MySymbol('R_c',unit=u.ohm,description='Widerstand zwischen der Versorgungsspannung und dem Kollektor')
Re = MySymbol('R_e',description='Widerstand zwischen dem Emitter und der Masse')
U1 = MySymbol('U_1')
U2 = MySymbol('U_2')
UBE = MySymbol('U_BE')
Uc = MySymbol('U_C')
Ue = MySymbol('U_E')
Ubat = MySymbol('U_bat')
Uout = MySymbol('U_out')
Uin = MySymbol('U_in')
Ib = MySymbol('I_B')
Ic = MySymbol('I_C')
Iq = MySymbol('I_Q')


### Die Emitterschaltung mit Temperaturstabilisierung
Ein einfacher Spannungsverstärker. Der Re dient der Temperaturstabilisierung der $U_{BE}$ Strecke.

![Bipolartransistor in Emittergrundschaltung mit Re](Grafiken/BJTEmitterschaltung_mitRE){#fig-BJT_Emitter_mit_Re}


### Aufgabenstellung
Entwerfen Sie einen Spannungsverstärker mit einer Verstärkung.

In [ ]:
#| code-fold: true
#| echo: false
vU = MySymbol('v_U',description='Spannungsverstärkung',value=-20)

vU_eq = Eq(vU,-Rc/Re)

QBookHelpers.print_equation(vU_eq,label='eq-vU_eq')
#QBookHelpers.print_description(vU_eq.free_symbols)
QBookHelpers.print_values([vU])

#### Gegeben

##### Aus der Angabe

In [ ]:
#| code-fold: true
#| echo: false
vU = MySymbol('v_U',description='Spannungsverstärkung',value=-20)
QBookHelpers.print_values([vU])
#QBookHelpers.print_description([vU])

Ubat = MySymbol('U_bat',description='Versorgungsspannung',value=10,unit=u.V)
QBookHelpers.print_values([Ubat])
#QBookHelpers.print_description([Ubat])

##### Aus dem Datenblatt

In [ ]:
#| code-fold: true
#| echo: false
B = MySymbol('B',description='Stromverstärkung, $h_{fe}$',value=300)

QBookHelpers.print_values([B])
#QBookHelpers.print_description([B])

##### Aus der Erfahrung / Faustregel
* Zahlenwerte

In [ ]:
#| code-fold: true
#| echo: false

Ut = MySymbol('U_T',description='Temperaturspannung',value=25*10**-3,unit=u.V)
QBookHelpers.print_values([Ut])
#QBookHelpers.print_description([Ut])

Ic = MySymbol('I_c',description='Strom in den Kollektor',value=1*10**-3,unit=u.A)
QBookHelpers.print_values([Ic])
#QBookHelpers.print_description([Ic])

Ube = MySymbol('U_BE',description='Spannungsabfall zwischen Basis und Emitter',value=0.7,unit=u.V)
QBookHelpers.print_values([Ube])
#QBookHelpers.print_description([Ube])


* Gleichungen

In [ ]:
#| code-fold: true
#| echo: false

Ic_eq = Eq(Ic,Ubat/(2*(Re+Rc)))

QBookHelpers.print_equation(Ic_eq,label='eq-Ic1')
#QBookHelpers.print_description(Ic_eq.free_symbols)

### Berechnung
* Gleichung @eq-vU_eq  nach $R_e$ auflösen. Das Ergebnis in Gleichung @eq-Ic1 einsetzen und nach $R_c$ auflösen.

In [ ]:
#| code-fold: true
#| echo: false

Re_eq  = Eq(Re,solve(vU_eq,Re)[0])
QBookHelpers.print_equation(vU_eq,label='eq-Re1')

Ic_eq2 = Ic_eq.subs(Re_eq.lhs,Re_eq.rhs)
QBookHelpers.print_equation(Ic_eq2,label='eq-Ic_eq2')

Rc_eq2  = Eq(Rc,solve(Ic_eq2,Rc)[0])
QBookHelpers.print_equation(Rc_eq2,label='eq-Rc1')

QBookHelpers.calculate_num_value(Rc_eq2)
QBookHelpers.print_values([Rc])

#QBookHelpers.print_description(Rc_eq2.free_symbols)

### Die Kollektorschaltung
Die Kollektorschaltung wird als Impedanzwandler eingesetzt. Daher ist die wesentliche Eigenschaft der Eingangswiderstand. Die Bauteile müssen also so berechnet werden, dass sich ein gewünschter Eingangwiderstand einstellt.


In [ ]:
#| code-fold: true
#| echo: false


**To be Continued**
